## Compute driving distances - IN PROGRESS
Here we modify our previous analysis that computed the Euclidean distance between candidate site (exits) and existing infrastructure (DCFC) by instead measuring *driving* distances. To do this we need a network dataset, often called a *graph dataset* of roads, which we get from OpenStreetMap, and a means for analyzing network datasets, which we can do via the `osmnx` package. 

The analysis here is guided by the analysis presented in [Lesson 6: Network Analysis in Python](https://automating-gis-processes.github.io/site/notebooks/L6/network-analysis.html) of the Automating GIS curriculum, but with a number of twists. We also need to dig deeper into the graph analysis capabilities of the [NetworkX package](https://networkx.github.io/documentation/networkx-1.10/index.html), specifically using its [shortest paths](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.shortest_paths.html) algorithms to measure network distances away from DFCF chargers. 

Prior to running this notebook, you should have already run `A4-Fetch-NC-roads-as-graph.ipynb` to download the NC roads as a graphml dataset into your Data/OSM folder.

The workflow here is:
* Import major roads as a graph dataset from OSM saved graphml file
* Import the DCFC csv file and convert to a geopandas dataframe
* Import the exits feature class as a geopdandas dataframe
* Compute weighted distance from each node to all other nodes capped at 100 miles
* From the above: compute distances from DCFC node to each Exit node
* Append node distances to exit features in exit geodataframe

The result will be a geodataframe with a new column indicating driving distances to the nearest DCFC, up to 100 miles, which is saved to a new shapefile: `exits_drivedist.shp`

In [ ]:
#Import packages
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

### Load data into the coding environment

In [ ]:
#Load in the NC road network
nc_graph = ox.load_graphml('NC_roads.graphml',folder='../Data/OSM/')

In [ ]:
#Load in the DCFC locations as pandas dataframe
dcfc_df = pd.read_csv('../Data/NREL/DCFC.csv')

In [ ]:
#Convert to a geopandas dataframe
geom_points = [Point(xy) for xy in zip(dcfc_df['longitude'],dcfc_df['latitude'])]
dcfc_gdf = gpd.GeoDataFrame(dcfc_df,geometry=geom_points,crs=4326)

In [ ]:
#Load exits as a geopandas dataframe
exits_gdf = gpd.read_file('../Data/MJBA/Exits.shp')

### Compute network node ids for each DCFC and Exit locations

In [ ]:
#Define the function to extract the nearest node ID for a point object
def get_nodeid(thePoint):
    #Get the yx tuple
    yx = (thePoint.y,thePoint.x)
    #Fetch the node nearest the xy tuple
    node_id = ox.get_nearest_node(nc_graph,yx)
    #Return the node id
    return node_id

In [ ]:
#Apply the function to each point in the DCFC geodataframe, adding node ID as a column
dcfc_gdf['node_id'] = dcfc_gdf['geometry'].apply(get_nodeid)

In [ ]:
#Apply the function to each point in the Exits geodataframe, adding node ID as a column
exits_gdf['node_id'] = exits_gdf['geometry'].apply(get_nodeid)

In [ ]:
#Create lists from the values, for later use
nodes_dcfc = dcfc_gdf['node_id'].unique()
nodes_exits = exits_gdf['node_id'].unique()

### Compute the distance away from each DCFC
With our network dataset, we can use the `networkx` package to compute distances along the network. This is done using the [Shortest Paths algorithms](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.shortest_paths.html), specifically the Dijkstra algorithms which computes a weighted distance (using lengths as our weight).  

We'll explore this analysis using the example of a single DCFC location, computing the distance of each node away from this point, and then assigning this distance to each Exit feature by determining the node that is nearest the Exit. The workflow is summarized as follows:
* Extract a single node ID (from the list generatd above)
* Use the `single_source_dijkstra_path_length()` algorithm to compute the network distance from this node to all other nodes in the graph, up to a limit of 100 miles. 
* From this result, extract the node nearest each exit and assign the distance to that node

In [ ]:
#Get the graph node nearest the point
theStartNode = nodes_dcfc[0]
theStartNode

In [ ]:
#Compute distance to other nodes, with cutoff of 100 miles
theSPs = nx.single_source_dijkstra_path_length(nc_graph,
                                               theStartNode,cutoff=100*1609.34,
                                               weight='length')

The result of the algorithm is a dictionary of nodes:distance values. A quick detour to convert the dictionary into a dataframe and plot the histogram of each node's distance away from the DCFC point.

In [ ]:
#Create a dataframe of the distances, and plot in miles
df = pd.DataFrame()
df['node_id']=theSPs.keys()
df['distance_m'] = theSPs.values()
df['distance_mi'] = df.distance_m/1609.34
df.distance_mi.hist();

In [ ]:
#Compute the paths to other nodes, with cutoff of 100 miles
thePaths = nx.single_source_dijkstra_path(nc_graph,
                                          theStartNode,cutoff=100*1609.34,
                                          weight='length')
type(thePaths)

This returns a dictionary listing all the nodes within 100 miles of a given node. So, we can iterate thro

In [ ]:
thePaths[theStartNode]

In [ ]:
nx.dijkstra_path_length(nc_graph,nodes_dcfc[0],nodes_exits[0],weight='length')

Now we boost the process to compute network distances from all DCFC locations, done by using the `all_pairs_dijkstra_path_length()` algorithm ([link](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.shortest_paths.weighted.single_source_dijkstra_path.html#single-source-dijkstra-path)), again applying a cutoff of 100 miles. 

In [ ]:
#Compute all pairs lengths
allDistPairs = nx.all_pairs_dijkstra_path_length(nc_graph,cutoff=100*1609.34,weight='length')
type(allDistPairs)

Because this function returns a large number of results, it returns a "generator" object, not a dictionary or list. The items in a generator object can only be accessed sequentially, so wee need to iterate through the objects to work with it. 

Below, we loop through all items in this generator. Each item in the generator is a tuple, with the first item in this tuple being the ID of the source node, and the second item is a dictionary containing each node falling within 100 miles and its distance from the source node. For example, the result of the first item would be:
```
(1859256320, {1859256320: 0, 149376304: 42.997, 149412495: 59.782000000000004})
```
Here, `1859256320` is the source node, corresponding to the DCFC location. And `{1859256320: 0, 149376304: 42.997, 149412495: 59.782000000000004}` reveals the connected nodes and their distances (in meters). 

So, in the code below, we iterate through each item in the generator, and at each iteration, we check to see if the start node corresponds to a DCFC location. If it does, then we investigate whether its connected nodes include any Exit nodes. And if it does, then we see if the distance between these two nodes is the minimum recorded distance. If it is, then we label that in the Edges dataframe; if not, then we keep the exising value...

In [ ]:
#Loop through the results and extract the distances associated with DCFC nodes
distance_data = {}
for distItem in allDistPairs:
    #Get the items in the tuple
    startNode_id = distItem[0]
    path_data = distItem[1]
    #Check whether the start node occurs in the list of DCFC nodes
    if startNode_id in nodes_dcfc:
        #If so, get its end nodes as a Python set object...
        endNodes = set(path_data.keys())
        #Now intersect this set with the set of exit node IDs
        valid_endNodes = endNodes.intersection(set(nodes_exits))
        #See if the intersection has any members
        if len(valid_endNodes) > 0:
            #If so, loop through items
            for node in valid_endNodes:
                #Get the distance associated with the node
                theDistance = path_data[node]
                #See if node already in dict
                if node in distance_data.keys():
                    #If so, compare existing distance to new distance
                    if distance_data[node] >= theDistance:
                        #If it's greater, update with smaller
                        distance_data[node] = theDistance
                #And if the node has not yet been added, add it
                else:
                    distance_data[node] = theDistance

In [ ]:
#Convert to a dataframe
df_distance = pd.DataFrame()
df_distance['node_id'] = distance_data.keys()
df_distance['dist_m'] = distance_data.values()
df_distance['dist_mi'] = df_distance['dist_m'] / 1609.34
df_distance.head()

In [ ]:
#Join the data to the exits data
exits_gdf1 = pd.merge(exits_gdf,df_distance,left_on='node_id',right_on='node_id',how='left')

In [ ]:
exits_gdf1.to_file('../Data/processed/exits_distance.shp')

### A bit of visualization...

In [ ]:
#Plot a histogram of distances
exits_gdf1['dist_mi'].hist();

In [ ]:
#Create a list of exit colors, for plotting
exit_colors = []
for d in exits_gdf1['dist_mi']:
    if d < 50: exit_colors.append('grey')
    elif d > 100: exit_colors.append('green')
    else: exit_colors.append('red')

In [ ]:
#Plot exits and charger locations
ax = exits_gdf1.plot(markersize='dist_mi',  #Make further exits larger
                     figsize=(20,15),       #Size of figure
                     color=exit_colors,     #Use colors set above
                     alpha=0.3)             #Set to mostly transparent
ax.set_title("NC Candidate Locations")

#Add the DCFC locations as small blue crosses
dcfc_gdf.plot(color='blue',marker='+',ax=ax);